In [1]:
from qulacs import QuantumCircuit, QuantumCircuitSimulator, ParametricQuantumCircuit, QuantumState
import numpy as np
import networkx as nx

In [6]:
##  RE-DEFINE SUBROUTINES  ##

def get_tsp_init_circuit(G: nx.Graph , init_state=None, encoding="onehot"):
    """
    Generates an inti state.

    Parameters
    ----------
    G : networkx.Graph
        Graph to solve TSP on
    init_state : list of integers

    Returns
    -------
    qc : ParametricQuantumCircuit
        Quantum circuit implementing the TSP phase unitary
    """
    if encoding == "onehot" and init_state:
        N = G.number_of_nodes()
        assert N == len(init_state)
        qc = ParametricQuantumCircuit(N**2)
        for i in range(N):
            qc.add_X_gate(i*N + init_state[i])
        return qc

    elif encoding == "onehot":
        N = G.number_of_nodes()
        qc = ParametricQuantumCircuit(N**2)
        for i in range(N):
            qc.add_X_gate(i*N+i)
        return qc

In [7]:
def get_ordering_swap_partial_mixing_circuit(
    G, i, j, u, v, beta, T, encoding="onehot", structure="pauli rotations"):
    """
    Generates an ordering swap partial mixer for the TSP mixing unitary.

    Parameters
    ----------
    G : networkx.Graph
        Graph to solve TSP on
    i, j :
        Positions in the ordering to be swapped
    u, v :
        Cities to be swapped
    beta :
        QAOA angle
    T :
        Number of Trotter steps
    encoding : string, default "onehot"
        Type of encoding for the city ordering

    Returns
    -------
    qc : qiskit.QuantumCircuit
        Quantum circuit implementing the TSP phase unitary
    """
    if encoding == "onehot" and structure == "pauli rotations":
        N = G.number_of_nodes()
        dt = beta/T
        qc = ParametricQuantumCircuit(N**2)
        qui = (N*i + u)
        qvj = (N*j + v)
        quj = (N*j + u)
        qvi = (N*i + v)
        for t in range(T):
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xxxx")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,1,1,1], dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, -dt, "xxyy")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,1,2,2], -dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xyxy")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,2,1,2], dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "xyyx")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [1,2,2,1], dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yxxy")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,1,1,2], dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yxyx")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,1,2,1], dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, -dt, "yyxx")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,2,1,1], -dt)
            # append_4_qubit_pauli_rotation_term(qc, qui, qvj, quj, qvi, dt, "yyyy")
            qc.add_parametric_multi_Pauli_rotation_gate([qui, qvj, quj, qvi], [2,2,2,2], dt)
        return qc

def get_simultaneous_ordering_swap_mixer(G, beta, T1, T2, encoding="onehot"):
    if encoding == "onehot":
        N = G.number_of_nodes()
        dt = beta/T2
        qc = ParametricQuantumCircuit(N**2)
        for t in range(T2):
            for i in range(N-1):
                for u, v in G.edges:
                    qc.merge_circuit(get_ordering_swap_partial_mixing_circuit(
                                G, i, i+1, u, v, dt, T1, encoding="onehot"))
            for u, v in G.edges:
                qc.merge_circuit(get_ordering_swap_partial_mixing_circuit(
                                G, N-1, 0, u, v, dt, T1, encoding="onehot"))
        return qc

In [8]:
def get_tsp_cost_operator_circuit(
    G, gamma, pen, encoding="onehot", structure= "zz rotation"):
    """
    Generates a circuit for the TSP phase unitary with optional penalty.

    Parameters
    ----------
    G : networkx.Graph
        Graph to solve TSP on
    gamma :
        QAOA parameter gamma
    pen :
        Penalty for edges with no roads
    encoding : string, default "onehot"
        Type of encoding for the city ordering
    translate :
        dictionary with city encoding (ascending numerical to problem encoding)

    Returns
    -------
    qc : qiskit.QuantumCircuit
        Quantum circuit implementing the TSP phase unitary
    """
    if encoding == "onehot" and structure == "zz rotation":
        N = G.number_of_nodes()
        if not nx.is_weighted(G):
            raise ValueError("Provided graph is not weighted")
        qc = ParametricQuantumCircuit(N**2)
        for n in range(N): # cycle over all cities in the input ordering
            for u in range(N):
                for v in range(N): #road from city v to city u
                    q1 = (n*N + u) % (N**2)
                    q2 = ((n+1)*N + v) % (N**2)
                    if G.has_edge(u, v):
                        # append_zz_term(qc, q1, q2, gamma * G[u][v]["weight"])
                        qc.add_parametric_multi_Pauli_rotation_gate([q1,q2], [3,3], gamma * G[u][v]["weight"])
                    else:
                        # append_zz_term(qc, q1, q2, gamma * pen)
                        qc.add_parametric_multi_Pauli_rotation_gate([q1,q2], [3,3], gamma * pen)
        return qc
    if encoding == "onehot" and structure == "controlled z":
        N = G.number_of_nodes()
        if not nx.is_weighted(G):
            raise ValueError("Provided graph is not weighted")
        qc = QuantumCircuit(N**2)
        for n in range(N): # cycle over all cities in the input ordering
            for u in range(N):
                for v in range(N): #road from city v to city u
                    q1 = (n*N + u) % (N**2)
                    q2 = ((n+1)*N + v) % (N**2)
                    if G.has_edge(u, v):
                        qc.crz(gamma * G[u][v]["weight"], q1, q2)
                    else:
                        qc.crz(gamma * pen, q1, q2)
        return qc


In [9]:
def get_tsp_qaoa_circuit(
    G, beta, gamma, T1=5, T2=5, pen=2,
    transpile_to_basis=True, save_state=True, encoding="onehot"
):
    if encoding == "onehot":
        
        assert len(beta) == len(gamma)
        p = len(beta)  # infering number of QAOA steps from the parameters passed
        N = G.number_of_nodes()
    
        
        # prepare the init state in onehot encoding
        qc = get_tsp_init_circuit(G, encoding="onehot")

        # second, apply p alternating operators
        for i in range(p):
            qc.merge_circuit(get_tsp_cost_operator_circuit(G, gamma[i], pen, encoding="onehot"))
            qc.merge_circuit(get_simultaneous_ordering_swap_mixer(G, beta[i], T1, T2, encoding="onehot"))
            
        # if transpile_to_basis:
        #     qc = transpile(qc, optimization_level=0, basis_gates=["u1", "u2", "u3", "cx"])
        # if save_state:
        #     qc.save_state()

        return qc





In [10]:
G = nx.complete_graph(3)
for (u,v,w) in G.edges(data=True):
    w['weight'] = np.random.randn()

tsp = get_tsp_qaoa_circuit(G, [0.1], [0.1])
print(tsp)

*** Quantum Circuit Info ***
# of qubit: 9
# of step : 1811
# of gate : 1830
# of 1 qubit gate: 3
# of 2 qubit gate: 27
# of 3 qubit gate: 0
# of 4 qubit gate: 1800
Clifford  : no
Gaussian  : no

*** Parameter Info ***
# of parameter: 1827



In [11]:
n = tsp.get_qubit_count()
state = QuantumState(n)

In [12]:
print(state)

 *** Quantum State ***
 * Qubit Count : 9
 * Dimension   : 512
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)


In [13]:
tsp.update_quantum_state(state)

In [14]:
print(state)

 *** Quantum State ***
 * Qubit Count : 9
 * Dimension   : 512
 * State vector : 
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      (0,0)
                      

In [66]:
p = [(tsp.get_parametric_gate_position(i), tsp.get_parameter(i)) for i in range(tsp.get_parameter_count())]

In [67]:
p

[(3, 0.2),
 (4, -0.061409723242656915),
 (5, -0.04730021142679785),
 (6, -0.061409723242656915),
 (7, 0.2),
 (8, 0.19123331647930963),
 (9, -0.04730021142679785),
 (10, 0.19123331647930963),
 (11, 0.2),
 (12, 0.2),
 (13, -0.061409723242656915),
 (14, -0.04730021142679785),
 (15, -0.061409723242656915),
 (16, 0.2),
 (17, 0.19123331647930963),
 (18, -0.04730021142679785),
 (19, 0.19123331647930963),
 (20, 0.2),
 (21, 0.2),
 (22, -0.061409723242656915),
 (23, -0.04730021142679785),
 (24, -0.061409723242656915),
 (25, 0.2),
 (26, 0.19123331647930963),
 (27, -0.04730021142679785),
 (28, 0.19123331647930963),
 (29, 0.2),
 (30, 0.004),
 (31, -0.004),
 (32, 0.004),
 (33, 0.004),
 (34, 0.004),
 (35, 0.004),
 (36, -0.004),
 (37, 0.004),
 (38, 0.004),
 (39, -0.004),
 (40, 0.004),
 (41, 0.004),
 (42, 0.004),
 (43, 0.004),
 (44, -0.004),
 (45, 0.004),
 (46, 0.004),
 (47, -0.004),
 (48, 0.004),
 (49, 0.004),
 (50, 0.004),
 (51, 0.004),
 (52, -0.004),
 (53, 0.004),
 (54, 0.004),
 (55, -0.004),
 (56, 